<a href="https://colab.research.google.com/github/cristianrovira/BTE320/blob/main/ISE441_Eurovision_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

import pulp as pl
import pandas as pd
from pulp import *

In [ ]:
file_name = "ISE441 - Eurovision Project (1).xlsx"

#Transportation sheets
plane_df = pd.read_excel(file_name, sheet_name="Plane Python")
boat_df = pd.read_excel(file_name, sheet_name="Boat Python")
train_df = pd.read_excel(file_name, sheet_name="Train Python")

#Accommodation sheets
hotel_df = pd.read_excel(file_name, sheet_name="Hotel Python")
hostel_df = pd.read_excel(file_name, sheet_name="Hostel Python")
airbnb_df = pd.read_excel(file_name, sheet_name="AirBNB Python")

#Cities
cities = sorted(hotel_df["City"].tolist())

#Arcs
arcs = [(i, j) for i in cities for j in cities if i != j]

#Sets
transportation_modes = ["plane", "boat", "train"]
accommodation_types = ["Hotel", "Hostel", "AirBNB"]

print("Cities: ", cities)
print("Number of cities: ", len(cities))
print("Transportation modes: ", transportation_modes)
print("Accommodation types: ", accommodation_types)

In [ ]:
#TRANSPORTATION

base_city = "Rome"

c={} #travel cost
u={} #mode availability

for i in cities:
    for j in cities:
        if i != j:
            for m in transportation_modes:
                c[(i, j, m)] = 0
                u[(i, j, m)] = 0

def fill_transport(sheet,mode_type):
  for _, row in sheet.iterrows():
    origin = str(row["Origin"])
    destination = str(row["Destination"])
    cost = row["Cost"]

    if origin in cities and destination in cities and origin != destination:
      if pd.isna(cost):
        u[(origin,destination,mode_type)] = 0
        c[(origin,destination,mode_type)] = 0
      else:
        u[(origin,destination,mode_type)] = 1
        c[(origin,destination,mode_type)] = float(cost)

#transportation Parameters
fill_transport(plane_df,"plane")
fill_transport(boat_df,"boat")
fill_transport(train_df,"train")

In [ ]:
#HOUSING ACCOMODATION

p={} #Accommodation price
v={} #Accommodation availability

for i in cities:
  for r in accommodation_types:
    p[(i,r)]=0
    v[(i,r)]=0

def fill_accomodation(sheet,accomodation_type):
  for _, row in sheet.iterrows():
    city = str(row["City"])
    cost = row["Cost"]
    if city in cities and not pd.isna(cost):
      p[(city,accomodation_type)]=float(cost)
      v[(city,accomodation_type)]=1

#accommodation Parameters
fill_accomodation(hotel_df,"Hotel")
fill_accomodation(hostel_df,"Hostel")
fill_accomodation(airbnb_df,"AirBNB")


In [ ]:
#Parameters
D=30 #Total number of days available for the trip
B=5000 #Budget
K=20 #Number of cities to vist

#Stay bounds per city
L = {i: 1 for i in cities} # minimum amount of days
U = {i: 5 for i in cities} #maximum amount of days


In [ ]:
print("Example travel cost: ", c[(cities[0],"plane")])
print("Example accommodation price: ", p[(cities[0],"Hotel")])

In [ ]:
#LP

#Create problem
prob = LpProblem("EurovisionTrip_Problem", LpMinimize)

#Decision Variables
x = LpVariable.dicts("x", arcs,0,1,cat="Binary")
y = LpVariable.dicts("y", [(i,j,m) for (i,j) in arcs for m in transportation_modes],0,1,cat="Binary")
z = LpVariable.dicts("z", cities, 0, 1, cat="Binary")
a = LpVariable.dicts("a", [(i,r) for i in cities for r in accommodation_types],0,1,cat="Binary")
t = LpVariable.dicts("t", [(i,r) for i in cities for r in accommodation_types],lowBound=0,cat="Continuous")
s = LpVariable.dicts("s", cities, lowBound=0, cat="Continuous")


In [ ]:
#Objective Function
prob += (lpSum(c[i,j,m])*y[(i,j,m)] for (i,j) in arcs for m in transportation_modes)+(lpSum(p[(i,r)]*t[(i,r)] for i in cities for r in accommodation_types))


In [ ]:
#Constraints

# 1. Visit exactly K cities
prob += lpSum(z[i] for i in cities) == K

# 2. Base city (visited)
prob += z[base_city] == 1

# 3. Flow conservation
for i in cities:
    # outgoing arcs from i
    prob += lpSum(x[(i, j)] for j in cities if i != j) == z[i]
    # incoming arcs to i
    prob += lpSum(x[(j, i)] for j in cities if i != j) == z[i]

# 4. subtour elimination
for i in cities:
    if i != base_city:
      #Lower bound
        prob += s[i] >= 2 * z[i]
      #Upper bound
        prob += s[i] <= K * z[i]

# link s and x to eliminate subtours
for i in cities:
    for j in cities:
        if i != j and i != base_city and j != base_city:
            prob += s[i] + K * x[(i, j)] + 1 <= s[j] + K

# 5. One mode per arc
for (i,j) in arcs:
  prob += lpSum(y[(i,j,m)] for m in transportation_modes) == x[(i,j)]

# 6. One accommodation per visited city
for i in cities:
  prob += lpSum(a[(i,r)] for r in accommodation_types) == z[i]

# 7. Stay time bounds
for i in cities:
  prob += lpSum(t[(i,r)] for r in accommodation_types) >= L[i] * z[i]
  prob += lpSum(t[(i,r)] for r in accommodation_types) <= U[i] * z[i]

# 8. Link days to accommodation
for i in cities:
    for r in accommodation_types:
        prob += t[(i,r)] <= U[i] * a[(i,r)]

# 6. Total days limit
prob += lpSum(t[(i,r)] for i in cities for r in accommodation_types) <= D

# 7. Budget limit
prob += (lpSum(c[(i,j,m)] * y[(i,j,m)] for (i,j) in arcs for m in transportation_modes) + lpSum(p[(i,r)] * t[(i,r)] for i in cities for r in accommodation_types)) <= B

# 8. Mode availability
for i in cities:
  for m in transportation_modes:
    prob += y[(i,m)] <= u[(i,m)]

# 9. Accommodation availability
for (i, in arcs:
  for r in accommodation_types:
    prob += a[(i,r)] <= v[(i,r)]


In [ ]:
print("Number of decision variables:", len(prob.variables()))  # number of variables
print("Number of constraints:", len(prob.constraints))  # number of constraints

# Solve LP
prob.solve()

# Print status
print("Status: ", LpStatus[prob.status])

if LpStatus[prob.status] == "Optimal":
  opt_z = value(prob.objective)
  print(f"Optimal Z: ${opt_z:,.2f}")
  print("\nVisited Cities: ")

  for i in cities:
    if x[i].value() ==1:
      chosen_modes = [m for m in transportation_modes if y[(i,m)].value() ==1]
      chosen_accommodations = [r for r in accommodation_types if a[(i,r)].value() ==1]
      days = sum(t[(i,r)].value() for r in accommodation_types)

      print("-",i)
      print("  Transportation mode used: ", chosen_modes)
      print("  Accommodation used: ", chosen_accommodations)
      print("  Stay time: ", days)
      print("------------------------------------------------")